In [15]:
import numpy as np
import pandas as pd
import folium

In [3]:
wells = pd.read_csv("../data/wells/AllWells_20210915.csv")

# Oil & Gas, Dry Gas (no liquids), Gas and "Liquefied Gas"
wells_oilgas = wells[ (wells['WellType'] == 'OG')  |
                      (wells['WellType'] == 'DG')  |
                      (wells['WellType'] == 'GAS') |
                      (wells['WellType'] == 'LG') ]
wells_latlon = wells_oilgas[['Latitude', 'Longitude']].to_records(index=False)
wells_oilgas.head()

,OID,API,LeaseName,WellNumber,WellStatus,WellType,WellTypeLa,OperatorCo,OperatorNa,FieldName,...,Township,Range,BaseMeridi,Latitude,Longitude,GISSource,isConfiden,isDirectio,SpudDate,WellSymbol
1,-1,402100016,Afton Community 1,1,Plugged,DG,Dry Gas,V1150,"Venoco, LLC",Afton Gas,...,19N,01W,MD,39.451515,-121.950668,hud,N,N,12/02/1943,PluggedDG
4,-1,402100017,Afton Community 2,1,Plugged,DG,Dry Gas,V1150,"Venoco, LLC",Afton Gas,...,19N,01W,MD,39.460217,-121.953537,hud,N,N,04/14/1945,PluggedDG
5,-1,402120283,Afton Community 2,2,Plugged,DG,Dry Gas,V1150,"Venoco, LLC",Afton Gas,...,19N,01W,MD,39.460033,-121.951424,hud,N,N,10/10/1984,PluggedDG
8,-1,402120815,Anacapa,3-4,Idle,DG,Dry Gas,R4085,"Royale Energy, Inc.",Afton Gas,...,18N,01W,MD,39.443230,-121.951042,GPS,N,N,11/06/2002,IdleDG
9,-1,402120494,Andreotti,15-26,Plugged,DG,Dry Gas,W1700,Western Continental Operating Company,Afton Gas,...,19N,01W,MD,39.470551,-121.947769,GPS,N,N,10/05/1987,PluggedDG


In [5]:
pooled_wells = pd.read_csv("../data/wells/pool_volumes.csv")
pwells_latlon = pooled_wells[['Latitude', 'Longitude']].to_records(index=False)
pwells_ids = pooled_wells['poolID'].tolist()
pwells_volumes = pooled_wells['totalco2'].tolist()
pooled_wells.head()

/Users/ashleymoss/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,poolID,Latitude,Longitude,WellDepth,gas2co2,oil2co2,totalco2
0,0,100001.0,37.693672,-121.689627,NaN,0.728255,1999.475066,2000.203320
1,1,120004.0,37.693731,-121.689699,NaN,27.734360,2820.538058,2848.272418
2,2,120005.0,37.695090,-121.689512,NaN,0.000000,1891.207349,1891.207349
3,3,120008.0,37.693752,-121.686563,NaN,0.000000,820.406824,820.406824
4,4,120009.0,37.694963,-121.686812,NaN,6.129476,3208.595801,3214.725276


In [11]:
emissions = pd.read_csv(r'../data/Emissions by Unit and Fuel Type - csv/UNIT_DATA-Table 1.csv')
emsCA2019 = emissions[emissions['Reporting Year'] == 2019]
facilities = pd.read_excel(r'../data/ghgp_data_2019.xlsx',
                          sheet_name='Direct Emitters', skiprows=3)
# One Michegan emitter lists its CA headquarters. Retag it as Michigan
facilities.loc[facilities['Facility Id'] == 1000594, 'State'] = 'MI'

facilities = facilities[(facilities.State == 'CA')]

facilitiesB = facilities.rename(inplace=False,
                                columns={'Facility Name':'facility_name',
                                         'City':'city',
                                         'Primary NAICS Code':'naics_code',
                                         'Industry Type (subparts)':'industry_subparts',
                                         'Industry Type (sectors)':'industry_sectors'})

emsCA2019loc = pd.merge(emsCA2019, facilitiesB[['Facility Id',
#                                               'Facility Name2',
                                               'city',
                                               'Zip Code',
                                               'Address',
                                               'County',
                                               'Latitude',
                                               'Longitude',
#                                               'Primary NAICS Code2',
                                               'industry_subparts',
                                               'industry_sectors']],
                        how='left', on='Facility Id')
emsCA2019locagg = emsCA2019loc[['Facility Id','Latitude','Longitude',
                                'Unit CO2 emissions (non-biogenic)']].dropna().groupby(
                                    'Facility Id').agg({'Latitude':'min',
                                                        'Longitude':'min',
                                                        'Unit CO2 emissions (non-biogenic)':'sum'})
emsCA2019locagg.reset_index(inplace=True)
emsCA2019locagg = emsCA2019locagg.rename(columns={'Facility Id':'facility_id',
                                                  'Latitude':'lat',
                                                  'Longitude':'lon',
                                                  'Unit CO2 emissions (non-biogenic)':'annual_emissions'})
emsCA2019locagg = emsCA2019locagg[emsCA2019locagg['annual_emissions'] > 0]
emsCA2019locagg.head()


,facility_id,lat,lon,annual_emissions
0,1000005,36.923750,-120.104120,60760.1
2,1000051,37.796500,-121.604900,52901.5
3,1000064,35.447778,-119.260000,64051.2
4,1000066,37.074790,-120.877445,69721.9
5,1000070,34.595300,-117.364700,1357906.4


,lat,lon
0,36.923750,-120.104120
1,37.796500,-121.604900
2,35.447778,-119.260000
3,37.074790,-120.877445
4,34.595300,-117.364700
...,...,...
274,37.498956,-120.897627
275,37.871889,-122.265978
276,37.528661,-122.026190
277,33.140110,-117.333660


In [24]:
pooled_wells.head()
#unique_suppliers


,Unnamed: 0,poolID,Latitude,Longitude,WellDepth,gas2co2,oil2co2,totalco2
0,0,100001.0,37.693672,-121.689627,NaN,0.728255,1999.475066,2000.203320
1,1,120004.0,37.693731,-121.689699,NaN,27.734360,2820.538058,2848.272418
2,2,120005.0,37.695090,-121.689512,NaN,0.000000,1891.207349,1891.207349
3,3,120008.0,37.693752,-121.686563,NaN,0.000000,820.406824,820.406824
4,4,120009.0,37.694963,-121.686812,NaN,6.129476,3208.595801,3214.725276


In [26]:
#create map
#find unique suppliers for looping and filtering
unique_suppliers = emsCA2019locagg[['lat', 'lon']]
unique_suppliers = unique_suppliers.drop_duplicates()
unique_suppliers =unique_suppliers.reset_index(drop=True)
unique_suppliers

#plot all the suppliers 
mapit = folium.Map( location=[37.3427, -119.2244], zoom_start=6)


#loop through suppliers, paint all the receivers in the same color
for i in range(0,unique_suppliers.shape[0]):
    
    #big circle for supplier
    folium.CircleMarker( location=[ unique_suppliers['lat'][i],unique_suppliers['lon'][i] ], fill=True, 
                        fill_color='blue',color='blue',  radius=1 ).add_to( mapit ) #'

#loop through suppliers, paint all the receivers in the same color
for i in range(0,pooled_wells.shape[0]):
    
    #big circle for supplier
    folium.CircleMarker( location=[ pooled_wells['Latitude'][i],pooled_wells['Longitude'][i] ], fill=True, 
                        fill_color='black',color='black',  radius=1 ).add_to( mapit ) #'

mapit

mapit.save(outfile= "all emitters.html")